# Bring Your Own Agent

# 1. Architecture

# 2. Bring Your Own Agent

## 2.1. Graph definition

### 2.1.1. Prompts

- coordinator

In [1]:
%%writefile ./src/prompts/coordinator.md

---
CURRENT_TIME: {CURRENT_TIME}
---

You are Bedrock-Manus, a friendly AI assistant developed by the Bedrock-Manus TF team (Dongjin Jang, Ph.D., AWS AIML Specialist SA).
You specialize in handling greetings and small talk, while directing complex tasks to a specialized planner.

# Details

Your primary responsibilities are:
- Introducing yourself as Bedrock-Manus when appropriate
- Responding to greetings (e.g., "hello", "hi", "good morning")
- Engaging in small talk (e.g., weather, time, how are you)
- Politely rejecting inappropriate or harmful requests
- Directing all other questions to the planner

# Execution Rules

- If the input is a greeting, small talk, or poses a security/moral risk:
  - Respond in plain text with an appropriate greeting or polite rejection
- For all other inputs:
  - Indicate that you need to pass this request to the planner by responding with:
  "handoff_to_planner: I'll need to consult our planning system for this request."

# Notes

- Always identify yourself as Bedrock-Manus when relevant
- Keep responses friendly but professional
- Don't attempt to solve complex problems or create plans yourself
- Always direct non-greeting queries to the planner
- Maintain the same language as the user

Overwriting ./src/prompts/coordinator.md


- coder

In [2]:
%%writefile ./src/prompts/coder.md

---
CURRENT_TIME: {CURRENT_TIME}
USER_REQUEST: {USER_REQUEST}
FULL_PLAN: {FULL_PLAN}
---

As a professional software engineer proficient in both Python and bash scripting, your mission is to analyze requirements, implement efficient solutions using Python and/or bash, and provide clear documentation of your methodology and results.

**[CRITICAL]** YOU ARE STRICTLY FORBIDDEN FROM: Creating PDF files (.pdf), HTML report files (.html), generating final reports or summaries, using weasyprint/pandoc or any report generation tools, or creating any document that resembles a final report. PDF/HTML/Report generation is EXCLUSIVELY the Reporter agent's job - NEVER YOURS! Execute ONLY the subtasks assigned to "Coder" in FULL_PLAN. Do NOT attempt to fulfill the entire USER_REQUEST - focus solely on your assigned coding/analysis tasks.

<steps>
1. Requirements Analysis: Carefully review the task description to understand the goals, constraints, and expected outcomes.
2. Solution Planning: 
   - [CRITICAL] Always implement code according to the provided FULL_PLAN (Coder part only)
   - Determine whether the task requires Python, bash, or a combination of both
   - Outline the steps needed to achieve the solution
3. Solution Implementation:
   - Use Python for data analysis, algorithm implementation, or problem-solving.
   - Use bash for executing shell commands, managing system resources, or querying the environment.
   - Seamlessly integrate Python and bash if the task requires both.
   - Use `print(...)` in Python to display results or debug values.
4. Solution Testing: Verify that the implementation meets the requirements and handles edge cases.
5. Methodology Documentation: Provide a clear explanation of your approach, including reasons for choices and assumptions made.
6. Results Presentation: Clearly display final output and intermediate results as needed.
   - Clearly display final output and all intermediate results
   - Include all intermediate process results without omissions
   - [CRITICAL] Document all calculated values, generated data, and transformation results with explanations at each intermediate step
   - [CRITICAL] **IMMEDIATE RECORDING AFTER EACH ANALYSIS**: Every time you complete ONE individual analysis step from the FULL_PLAN, you MUST immediately save the results to './artifacts/all_results.txt' before moving to the next analysis
   - [REQUIRED] Do NOT wait until all analyses are complete - record each analysis IMMEDIATELY upon completion to preserve rich details
   - Create the './artifacts' directory if no files exist there, or append to existing files
   - Record important observations discovered during the process
   - This prevents loss of detailed insights and ensures comprehensive documentation
</steps>

<data_analysis_requirements>
- [CRITICAL] Always explicitly read data files before any analysis:
  1. For any data analysis, ALWAYS include file reading step FIRST
  2. NEVER assume a DataFrame ('df' or any other variable) exists without defining it
  3. ALWAYS use the appropriate reading function based on file type:
     - CSV: df = pd.read_csv('path/to/file.csv')
     - Parquet: df = pd.read_parquet('path/to/file.parquet')
     - Excel: df = pd.read_excel('path/to/file.xlsx')
     - JSON: df = pd.read_json('path/to/file.json')
  4. Include error handling for file operations when appropriate

- [REQUIRED] Data Analysis Checklist (verify before executing any code):
  - [ ] All necessary libraries imported (pandas, numpy, etc.)
  - [ ] File path clearly defined (as variable or direct parameter)
  - [ ] Appropriate file reading function used based on file format
  - [ ] DataFrame explicitly created with reading function

- [EXAMPLE] Correct approach:
```python
import pandas as pd
import numpy as np

# Define file path
file_path = 'data.csv'  # Always define the file path

# Explicitly read the file and create DataFrame
df = pd.read_csv(file_path)  # MUST define the DataFrame

# Now perform analysis
print("Data overview:")
print(df.head())
print(df.describe())
```
</data_analysis_requirements>
 
<matplotlib_requirements>
- [CRITICAL] Must declare one of these matplotlib styles when you use `matplotlib`:
    - plt.style.use(['ipynb', 'use_mathtext','colors5-light']) - Notebook-friendly style with mathematical typography and a light color scheme with 5 distinct colors
    - plt.style.use('ggplot') - Clean style suitable for academic publications
    - plt.style.use('seaborn-v0_8') - Modern, professional visualizations
    - plt.style.use('fivethirtyeight') - Web/media-friendly style
- [CRITICAL] Must import lovelyplots at the beginning of visualization code:
    - import lovelyplots  # Don't omit this import
- **[CRITICAL] Use Korean font settings (REQUIRED for Korean text display):**
  ```python
  # Korean font setup - ALWAYS include this code
  plt.rc('font', family='NanumGothic')
  plt.rcParams['axes.unicode_minus'] = False  # Fix minus sign display
  
  # Alternative fonts if NanumGothic fails:
  # plt.rc('font', family=['NanumBarunGothic', 'NanumGothic', 'Malgun Gothic', 'DejaVu Sans'])
  ```
- Apply grid lines to all graphs (alpha=0.3)
- DPI: 150 (high resolution)
- Set font sizes: title: 14-16, axis labels: 12-14, tick labels: 8-10, legend: 8-10
- Use subplot() when necessary to compare related data
- [EXAMPLE] is below:

```python
# Correct visualization setup - ALWAYS USE THIS PATTERN
import matplotlib.pyplot as plt
import lovelyplots  # [CRITICAL] ALWAYS import this

# [CRITICAL] ALWAYS set a style
plt.style.use(['ipynb', 'use_mathtext','colors5-light'])  # Choose one from the required styles

# Set Korean font and other required parameters
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False  # Fix minus sign display
plt.figure(figsize=(10, 6), dpi=150)

# Rest of visualization code
```
</matplotlib_requirements>

<cumulative_result_storage_requirements>
- [CRITICAL] **EXECUTE THIS CODE AFTER EACH INDIVIDUAL ANALYSIS COMPLETION**: Every time you finish ONE analysis step from the FULL_PLAN, immediately run the result storage code below.
- [REQUIRED] **DO NOT BATCH MULTIPLE ANALYSES**: Save each analysis individually and immediately to preserve detailed insights.
- Always accumulate and save to './artifacts/all_results.txt'. Do not create other files.
- Do not omit `import pandas as pd`.
- [CRITICAL] Always include key insights and discoveries for Reporter agent to use.
- **WORKFLOW**: Complete Analysis 1 → Save to all_results.txt → Complete Analysis 2 → Save to all_results.txt → etc.
- Example is below:

```python
# Result accumulation storage section
import os
import pandas as pd
from datetime import datetime

# Create artifacts directory
os.makedirs('./artifacts', exist_ok=True)

# Result file path
results_file = './artifacts/all_results.txt'
backup_file = './artifacts/all_results_backup_{{}}.txt'.format(datetime.now().strftime("%Y%m%d_%H%M%S"))

# Current analysis parameters - modify these values according to your actual analysis
stage_name = "Analysis_Stage_Name"
result_description = """Description of analysis results
Also add actual analyzed data (statistics, distributions, ratios, etc.)
Can be written over multiple lines.
Include result values."""

# [CRITICAL] Key findings and insights from analysis - ALWAYS include this section
key_insights = """
[DISCOVERY & INSIGHTS]:
- Discovery 1: What patterns or anomalies did you find in the data?
- Insight 1: What does this discovery mean for the business/domain?
- Discovery 2: Any unexpected correlations or trends?
- Insight 2: How does this impact decision-making or understanding?
- Methodology insight: Why did you choose this analysis approach?
- Business implication: What actions or further investigations are recommended?
"""

artifact_files = [
    ## Always use paths that include './artifacts/' 
    ["./artifacts/generated_file1.extension", "File description"],
    ["./artifacts/generated_file2.extension", "File description"]
]

# Direct generation of result text without using a function
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
current_result_text = """
==================================================
## Analysis Stage: {{0}}
## Execution Time: {{1}}
--------------------------------------------------
Result Description: 
{{2}}
--------------------------------------------------
Key Findings & Insights:
{{3}}
""".format(stage_name, current_time, result_description, key_insights)

if artifact_files:
    current_result_text += "--------------------------------------------------\nGenerated Files:\n"
    for file_path, file_desc in artifact_files:
        current_result_text += "- {{}} : {{}}\n".format(file_path, file_desc)

current_result_text += "==================================================\n"

# Backup existing result file and accumulate results
if os.path.exists(results_file):
    try:
        # Check file size
        if os.path.getsize(results_file) > 0:
            # Create backup
            with open(results_file, 'r', encoding='utf-8') as f_src:
                with open(backup_file, 'w', encoding='utf-8') as f_dst:
                    f_dst.write(f_src.read())
            print("Created backup of existing results file: {{}}".format(backup_file))
    except Exception as e:
        print("Error occurred during file backup: {{}}".format(e))

# Add new results (accumulate to existing file)
try:
    with open(results_file, 'a', encoding='utf-8') as f:
        f.write(current_result_text)
    print("Results successfully saved.")
except Exception as e:
    print("Error occurred while saving results: {{}}".format(e))
    # Try saving to temporary file in case of error
    try:
        temp_file = './artifacts/result_emergency_{{}}.txt'.format(datetime.now().strftime("%Y%m%d_%H%M%S"))
        with open(temp_file, 'w', encoding='utf-8') as f:
            f.write(current_result_text)
        print("Results saved to temporary file: {{}}".format(temp_file))
    except Exception as e2:
        print("Temporary file save also failed: {{}}".format(e2))
```
</cumulative_result_storage_requirements>

<code_saving_requirements>
- [CRITICAL] When the user requests "write code", "generate code", or similar:
  - All generated code files must be saved to the "./artifacts/" directory
  - Always include code to check if the directory exists and create it if necessary
  - Always use clearly defined file paths that start with "./artifacts/"
  - Always include the actual code to save the file

- Example:
```python
import os

# Create artifacts directory
os.makedirs("./artifacts", exist_ok=True)

# Save code file
with open("./artifacts/solution.py", "w") as f:
    f.write("""
# Generated code content here
def main():
    print("Hello, world!")

if __name__ == "__main__":
    main()
""")

print("Code has been saved to ./artifacts/solution.py")
```
</code_saving_requirements>

<note>

- Always ensure that your solution is efficient and follows best practices.
- Handle edge cases gracefully, such as empty files or missing inputs.
- Use comments to improve readability and maintainability of your code.
- If you want to see the output of a value, you must output it with print(...).
- Always use Python for mathematical operations.
- [CRITICAL] Do not generate Reports or PDF files. Reports and PDF generation are STRICTLY the responsibility of the Reporter agent.
- [FORBIDDEN] Never create final reports, summary documents, or PDF files even if it seems logical or the plan is unclear.
- Always use yfinance for financial market data:
  - Use yf.download() to get historical data
  - Access company information with Ticker objects
  - Use appropriate date ranges for data retrieval
- **Package Installation (REQUIRED)**: 
  - [CRITICAL] When you need additional Python packages that are not already available, use UV package manager:
    ```bash
    uv add package-name
    ```
  - [EXAMPLES] Common package installations:
    ```bash
    uv add pandas numpy matplotlib seaborn scikit-learn
    ```
  - [FORBIDDEN] **NEVER use pip install** - always use `uv add` for package installation
  - [IMPORTANT] After installing with `uv add`, the package is immediately available in subsequent Python code executions
- Pre-installed packages in current environment:
  - pandas for data manipulation
  - numpy for numerical operations
  - matplotlib, seaborn for visualization
  - scikit-learn for machine learning
  - boto3 for AWS services
- Save all generated files and images to the ./artifacts directory:
  - Create this directory if it doesn't exist with os.makedirs("./artifacts", exist_ok=True)
  - Use this path when writing files, e.g., plt.savefig("./artifacts/plot.png")
  - Specify this path when generating output that needs to be saved to disk
- [CRITICAL] Always write code according to the plan defined in the FULL_PLAN (Coder part only) variable
- [CRITICAL] Always analyze the entire USER_REQUEST to detect the main language and respond in that language. For mixed languages, use whichever language is dominant in the request.
</note>

Overwriting ./src/prompts/coder.md


### 2.1.2 Tools

- python_repl_tool

In [3]:
%%writefile ./src/tools/python_repl_tool.py

import os
import sys
import time
import logging
import subprocess
from typing import Any, Annotated
from strands.types.tools import ToolResult, ToolUse
from src.tools.decorators import log_io

# Observability
from opentelemetry import trace
from src.utils.agentcore_observability import add_span_event

# Simple logger setup
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

TOOL_SPEC = {
    "name": "python_repl_tool",
    "description": "Use this to execute python code and do data analysis or calculation. If you want to see the output of a value, you should print it out with `print(...)`. This is visible to the user.",
    "inputSchema": {
        "json": {
            "type": "object",
            "properties": {
                "code": {
                    "type": "string",
                    "description": "The python code to execute to do further analysis or calculation."
                }
            },
            "required": ["code"]
        }
    }
}

class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

class PythonREPL:
    def __init__(self):
        pass
        
    def run(self, command):
        try:
            # 입력된 명령어 실행
            result = subprocess.run(
                [sys.executable, "-c", command],
                capture_output=True,
                text=True,
                timeout=600  # 타임아웃 설정
            )
            # 결과 반환
            if result.returncode == 0:
                return result.stdout
            else:
                return f"Error: {result.stderr}"
        except Exception as e:
            return f"Exception: {str(e)}"

repl = PythonREPL()

@log_io
def handle_python_repl_tool(code: Annotated[str, "The python code to execute to do further analysis or calculation."]):
    
    """
    Use this to execute python code and do data analysis or calculation. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user.
    """
    tracer = trace.get_tracer(
        instrumenting_module_name=os.getenv("TRACER_MODULE_NAME", "insight_extractor_agent"),
        instrumenting_library_version=os.getenv("TRACER_LIBRARY_VERSION", "1.0.0")
    )
    with tracer.start_as_current_span("python_repl_tool") as span:
        print()  # Add newline before log
        logger.info(f"{Colors.GREEN}===== Executing Python code ====={Colors.END}")
        try:
            result = repl.run(code)
        except BaseException as e:
            error_msg = f"Failed to execute. Error: {repr(e)}"
            logger.debug(f"{Colors.RED}Failed to execute. Error: {repr(e)}{Colors.END}")
            
            # Add Event
            add_span_event(span, "code", {"code": str(code)})
            add_span_event(span, "result", {"response": repr(e)})
            
            return error_msg
        
        #result_str = f"Successfully executed:\n||```python\n{code}\n```\n||Stdout: {result}"
        result_str = f"Successfully executed:\n||{code}||{result}"
        logger.info(f"{Colors.GREEN}===== Code execution successful ====={Colors.END}")

        # Add Event
        add_span_event(span, "code", {"code": str(code)})
        add_span_event(span, "result", {"response": str(result)})
        
        return result_str

# Function name must match tool name
def python_repl_tool(tool: ToolUse, **kwargs: Any) -> ToolResult:
    tool_use_id = tool["toolUseId"]
    code = tool["input"]["code"]
    
    # Use the existing handle_python_repl_tool function
    result = handle_python_repl_tool(code)
    
    # Check if execution was successful based on the result string
    if "Failed to execute" in result:
        return {
            "toolUseId": tool_use_id,
            "status": "error",
            "content": [{"text": result}]
        }
    else:
        return {
            "toolUseId": tool_use_id,
            "status": "success",
            "content": [{"text": result}]
        }

Overwriting ./src/tools/python_repl_tool.py


- coder_agent_tool (agent as a tool)

In [4]:
%%writefile ./src/tools/coder_agent_tool.py

import os
import logging
import asyncio
from typing import Any, Annotated
from strands.types.tools import ToolResult, ToolUse
from src.utils.strands_sdk_utils import strands_utils
from src.prompts.template import apply_prompt_template
from src.utils.common_utils import get_message_from_string
from src.tools import python_repl_tool, bash_tool

# Observability
from opentelemetry import trace
from src.utils.agentcore_observability import add_span_event

# Simple logger setup
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

TOOL_SPEC = {
    "name": "coder_agent_tool",
    "description": "Execute Python code and bash commands using a specialized coder agent. This tool provides access to a coder agent that can execute Python code for data analysis and calculations, run bash commands for system operations, and handle complex programming tasks.",
    "inputSchema": {
        "json": {
            "type": "object",
            "properties": {
                "task": {
                    "type": "string",
                    "description": "The coding task or question that needs to be executed by the coder agent."
                }
            },
            "required": ["task"]
        }
    }
}

RESPONSE_FORMAT = "Response from {}:\n\n<response>\n{}\n</response>\n\n*Please execute the next step.*"
FULL_PLAN_FORMAT = "Here is full plan :\n\n<full_plan>\n{}\n</full_plan>\n\n*Please consider this to select the next step.*"
CLUES_FORMAT = "Here is clues from {}:\n\n<clues>\n{}\n</clues>\n\n"

class Colors:
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    END = '\033[0m'

def handle_coder_agent_tool(task: Annotated[str, "The coding task or question that needs to be executed by the coder agent."]):
    """
    Execute Python code and bash commands using a specialized coder agent.
    
    This tool provides access to a coder agent that can:
    - Execute Python code for data analysis and calculations
    - Run bash commands for system operations
    - Handle complex programming tasks
    
    Args:
        task: The coding task or question that needs to be executed
        
    Returns:
        The result of the code execution or analysis
    """
    tracer = trace.get_tracer(
        instrumenting_module_name=os.getenv("TRACER_MODULE_NAME", "insight_extractor_agent"),
        instrumenting_library_version=os.getenv("TRACER_LIBRARY_VERSION", "1.0.0")
    )
    with tracer.start_as_current_span("coder_agent_tool") as span:
        print()  # Add newline before log
        logger.info(f"\n{Colors.GREEN}Coder Agent Tool starting task{Colors.END}")
        
        # Try to extract shared state from global storage
        from src.graph.nodes import _global_node_states
        shared_state = _global_node_states.get('shared', None)
        
        if not shared_state:
            logger.warning("No shared state found")
            return "Error: No shared state available" 
                        
        request_prompt, full_plan = shared_state.get("request_prompt", ""), shared_state.get("full_plan", "")
        clues, messages = shared_state.get("clues", ""), shared_state.get("messages", [])
        
        # Create coder agent with specialized tools using consistent pattern
        coder_agent = strands_utils.get_agent(
            agent_name="coder",
            system_prompts=apply_prompt_template(prompt_name="coder", prompt_context={"USER_REQUEST": request_prompt, "FULL_PLAN": full_plan}),
            agent_type="claude-sonnet-3-7", # claude-sonnet-3-5-v-2, claude-sonnet-3-7
            enable_reasoning=False,
            tools=[python_repl_tool, bash_tool],
            #tools=[code_interpreter_tool],
            streaming=True  # Enable streaming for consistency
        )
        
        # Prepare message with context if available
        message = '\n\n'.join([messages[-1]["content"][-1]["text"], clues])
        
        # Process streaming response and collect text in one pass
        async def process_coder_stream():
            full_text = ""
            async for event in strands_utils.process_streaming_response_yield(
                coder_agent, message, agent_name="coder", source="coder_tool"
            ):
                if event.get("event_type") == "text_chunk": full_text += event.get("data", "")
            return {"text": full_text}
        
        response = asyncio.run(process_coder_stream())
        result_text = response['text']
        
        # Update clues
        clues = '\n\n'.join([clues, CLUES_FORMAT.format("coder", response["text"])])
        
        # Update history
        history = shared_state.get("history", [])
        history.append({"agent":"coder", "message": response["text"]})
        
        # Update shared state
        shared_state['messages'] = [get_message_from_string(role="user", string=RESPONSE_FORMAT.format("coder", response["text"]), imgs=[])]
        shared_state['clues'] = clues
        shared_state['history'] = history
        
        logger.info(f"\n{Colors.GREEN}Coder Agent Tool completed successfully{Colors.END}")

        # Add Event
        add_span_event(span, "input_message", {"message": str(message)})
        add_span_event(span, "response", {"response": str(response["text"])})

        return result_text

# Function name must match tool name
def coder_agent_tool(tool: ToolUse, **_kwargs: Any) -> ToolResult:
    tool_use_id = tool["toolUseId"]
    task = tool["input"]["task"]
    
    # Use the existing handle_coder_agent_tool function
    result = handle_coder_agent_tool(task)
    
    # Check if execution was successful based on the result string
    if "Error in coder agent tool" in result:
        return {
            "toolUseId": tool_use_id,
            "status": "error",
            "content": [{"text": result}]
        }
    else:
        return {
            "toolUseId": tool_use_id,
            "status": "success",
            "content": [{"text": result}]
        }

Overwriting ./src/tools/coder_agent_tool.py


### 2.1.3 Nodes
- 각 노드별 로직을 정의합니다.

In [5]:
%%writefile ./src/graph/nodes.py

import os
import logging
from src.utils.strands_sdk_utils import strands_utils
from src.prompts.template import apply_prompt_template
from src.utils.common_utils import get_message_from_string

# Tools
from src.tools import coder_agent_tool, reporter_agent_tool, tracker_agent_tool

# Observability
from opentelemetry import trace
from src.utils.agentcore_observability import add_span_event

# Simple logger setup
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

class Colors:
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    END = '\033[0m'

def log_node_start(node_name: str):
    """Log the start of a node execution."""
    print()  # Add newline before log
    logger.info(f"{Colors.GREEN}===== {node_name} started ====={Colors.END}")

def log_node_complete(node_name: str):
    """Log the completion of a node."""
    print()  # Add newline before log
    logger.info(f"{Colors.GREEN}===== {node_name} completed ====={Colors.END}")

# Global state storage for sharing between nodes
_global_node_states = {}

RESPONSE_FORMAT = "Response from {}:\n\n<response>\n{}\n</response>\n\n*Please execute the next step.*"
FULL_PLAN_FORMAT = "Here is full plan :\n\n<full_plan>\n{}\n</full_plan>\n\n*Please consider this to select the next step.*"
CLUES_FORMAT = "Here is clues from {}:\n\n<clues>\n{}\n</clues>\n\n"


def should_handoff_to_planner(_):
    """Check if coordinator requested handoff to planner."""

    tracer = trace.get_tracer(
        instrumenting_module_name=os.getenv("TRACER_MODULE_NAME", "insight_extractor_agent"),
        instrumenting_library_version=os.getenv("TRACER_LIBRARY_VERSION", "1.0.0")
    )
    with tracer.start_as_current_span("should_handoff_to_planner") as span:
        # Check coordinator's response for handoff request
        global _global_node_states
        shared_state = _global_node_states.get('shared', {})
        history = shared_state.get('history', [])
        
        # Look for coordinator's last message
        for entry in reversed(history):
            if entry.get('agent') == 'coordinator':
                message = entry.get('message', '')
                
                # Add Event
                add_span_event(span, "input_message", {"message": str(message)})
                add_span_event(span, "response", {"handoff_to_planner": bool("handoff_to_planner" in message)})

                return 'handoff_to_planner' in message
        
        return False

async def coordinator_node(task=None, **kwargs):
    
    tracer = trace.get_tracer(
        instrumenting_module_name=os.getenv("TRACER_MODULE_NAME", "insight_extractor_agent"),
        instrumenting_library_version=os.getenv("TRACER_LIBRARY_VERSION", "1.0.0")
    )
    with tracer.start_as_current_span("coordinator") as span:
        """Coordinator node that communicate with customers."""
        global _global_node_states
        
        log_node_start("Coordinator")

        # Extract user request from task (now passed as dictionary)
        if isinstance(task, dict):
            request = task.get("request", "")
            request_prompt = task.get("request_prompt", request)
        else:
            request = str(task) if task else ""
            request_prompt = request

        agent = strands_utils.get_agent(
            agent_name="coordinator",
            system_prompts=apply_prompt_template(prompt_name="coordinator", prompt_context={}), # apply_prompt_template(prompt_name="task_agent", prompt_context={"TEST": "sdsd"})
            agent_type="claude-sonnet-3-5-v-2", # claude-sonnet-3-5-v-2, claude-sonnet-3-7
            enable_reasoning=False,
            prompt_cache_info=(False, None), #(False, None), (True, "default")
            streaming=True,
        )
            
        # Process streaming response and collect text in one pass
        full_text = ""
        async for event in strands_utils.process_streaming_response_yield(
            agent, request_prompt, agent_name="coordinator", source="coordinator_node"
        ):
            if event.get("event_type") == "text_chunk": 
                full_text += event.get("data", "")
        response = {"text": full_text}
        
        # Store data directly in shared global storage
        if 'shared' not in _global_node_states: _global_node_states['shared'] = {}
        shared_state = _global_node_states['shared']
        
        # Update shared global state
        shared_state['messages'] = agent.messages
        shared_state['request'] = request
        shared_state['request_prompt'] = request_prompt
        
        # Build and update history
        if 'history' not in shared_state: 
            shared_state['history'] = []
        shared_state['history'].append({"agent":"coordinator", "message": response["text"]})
        
        # Add Event
        add_span_event(span, "input_message", {"message": str(request_prompt)})
        add_span_event(span, "response", {"response": str(response["text"])})

        log_node_complete("Coordinator")
        # Return response only
        return response

async def planner_node(task=None, **kwargs):
    
    tracer = trace.get_tracer(
        instrumenting_module_name=os.getenv("TRACER_MODULE_NAME", "insight_extractor_agent"),
        instrumenting_library_version=os.getenv("TRACER_LIBRARY_VERSION", "1.0.0")
    )
    with tracer.start_as_current_span("planner") as span:   
        """Planner node that generates detailed plans for task execution."""
        log_node_start("Planner")
        global _global_node_states
        
        # Extract shared state from global storage
        shared_state = _global_node_states.get('shared', None)
        
        # Get request from shared state (task parameter not used in planner)
        request = shared_state.get("request", "") if shared_state else ""
        
        if not shared_state:
            logger.warning("No shared state found in global storage")
            return None, {"text": "No shared state available"}

        agent = strands_utils.get_agent(
            agent_name="planner",
            system_prompts=apply_prompt_template(prompt_name="planner", prompt_context={"USER_REQUEST": request}),
            agent_type="claude-sonnet-3-7", # claude-sonnet-3-5-v-2, claude-sonnet-3-7
            enable_reasoning=True,
            prompt_cache_info=(False, None),  # enable prompt caching for reasoning agent, (False, None), (True, "default")
            streaming=True,
        )
        
        full_plan, messages = shared_state.get("full_plan", ""), shared_state["messages"]
        message = '\n\n'.join([messages[-1]["content"][-1]["text"], FULL_PLAN_FORMAT.format(full_plan)])
        
        # Process streaming response and collect text in one pass
        full_text = ""
        async for event in strands_utils.process_streaming_response_yield(
            agent, message, agent_name="planner", source="planner_node"
        ):
            if event.get("event_type") == "text_chunk": full_text += event.get("data", "")
        response = {"text": full_text}
        
        # Update shared global state
        shared_state['messages'] = [get_message_from_string(role="user", string=response["text"], imgs=[])]
        shared_state['full_plan'] = response["text"]
        shared_state['history'].append({"agent":"planner", "message": response["text"]})

        # Add Event
        add_span_event(span, "input_message", {"message": str(message)})
        add_span_event(span, "response", {"response": str(response["text"])})

        log_node_complete("Planner")
        # Return response only
        return response

async def supervisor_node(task=None, **kwargs):
    """Supervisor node that decides which agent should act next."""
    log_node_start("Supervisor")
    global _global_node_states

    # task and kwargs parameters are unused - supervisor relies on global state
    tracer = trace.get_tracer(
        instrumenting_module_name=os.getenv("TRACER_MODULE_NAME", "insight_extractor_agent"),
        instrumenting_library_version=os.getenv("TRACER_LIBRARY_VERSION", "1.0.0")
    )
    with tracer.start_as_current_span("supervisor") as span:  

        # Extract shared state from global storage
        shared_state = _global_node_states.get('shared', None)
        
        if not shared_state:
            logger.warning("No shared state found in global storage")
            return None, {"text": "No shared state available"}

        agent = strands_utils.get_agent(
            agent_name="supervisor",
            system_prompts=apply_prompt_template(prompt_name="supervisor", prompt_context={}),
            agent_type="claude-sonnet-3-7", # claude-sonnet-3-5-v-2, claude-sonnet-3-7
            enable_reasoning=False,
            prompt_cache_info=(True, "default"),  # enable prompt caching for reasoning agent
            tools=[coder_agent_tool, reporter_agent_tool, tracker_agent_tool],  # Add coder, reporter and tracker agents as tools
            streaming=True,
        )

        clues, full_plan, messages = shared_state.get("clues", ""), shared_state.get("full_plan", ""), shared_state["messages"]
        message = '\n\n'.join([messages[-1]["content"][-1]["text"], FULL_PLAN_FORMAT.format(full_plan), clues])
            
        # Process streaming response and collect text in one pass
        full_text = ""
        async for event in strands_utils.process_streaming_response_yield(
            agent, message, agent_name="supervisor", source="supervisor_node"
        ):
            if event.get("event_type") == "text_chunk": full_text += event.get("data", "")
        response = {"text": full_text}

        # Update shared global state
        shared_state['history'].append({"agent":"supervisor", "message": response["text"]})
        
        # Add Event
        add_span_event(span, "input_message", {"message": str(message)})
        add_span_event(span, "response", {"response": str(response["text"])})
        
        log_node_complete("Supervisor")
        logger.info("Workflow completed")
        # Return response only
        return response

Overwriting ./src/graph/nodes.py


### 2.1.4 Edges (Build Graph)
- Strans Agents - [Grpah Pattern](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/graph/?h=graph)

In [6]:
%%writefile ./src/graph/builder.py

from strands.multiagent import GraphBuilder
from src.utils.strands_sdk_utils import FunctionNode
from .nodes import (
    supervisor_node,
    coordinator_node,
    planner_node,
    should_handoff_to_planner,
)

def build_graph():
    """Build and return the agent workflow graph."""
    builder = GraphBuilder()

    # Nodes
    coordinator = FunctionNode(func=coordinator_node, name="coordinator")
    planner = FunctionNode(func=planner_node, name="planner")
    supervisor = FunctionNode(func=supervisor_node, name="supervisor")
    
    builder.add_node(coordinator, "coordinator")
    builder.add_node(planner, "planner")
    builder.add_node(supervisor, "supervisor")

    # Set entry points (optional - will be auto-detected if not specified)
    builder.set_entry_point("coordinator")
    
    # Add conditional edge - only go to planner if handoff is requested
    builder.add_edge("coordinator", "planner", condition=should_handoff_to_planner)
    
    # Add edge - planner to supervisor (no condition needed)
    builder.add_edge("planner", "supervisor")

    # Build the graph
    return builder.build()

Overwriting ./src/graph/builder.py


## 2.2. Workflow and Main defination

### 2.2.1 workflow

In [7]:
%%writefile ./src/workflow.py

import logging
from src.graph.builder import build_graph

# Simple logger setup
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

class Colors:
    GREEN = '\033[92m'
    END = '\033[0m'

async def run_graph_streaming_workflow(user_input: str):
    """Full graph streaming workflow that maintains graph structure.
    
    Args:
        user_input: The user's query or request  
        
    Returns:
        The result of the workflow execution
    """
    if not user_input:
        raise ValueError("Input could not be empty")

    logger.info(f"\n{Colors.GREEN}Starting graph streaming workflow{Colors.END}")
    
    # Prepare user prompt
    user_prompts = f"Here is a user request: <user_request>{user_input}</user_request>"


    #########################
    ## modification START  ##
    #########################

    # Build and execute graph
    graph = build_graph()
    result = await graph.invoke_async(
        task={
            "request": user_input,
            "request_prompt": user_prompts
        }
    )
    #########################
    ## modification END    ##
    #########################
    
    logger.info(f"\n{Colors.GREEN}Graph streaming workflow completed{Colors.END}")
    return result


Overwriting ./src/workflow.py


### 2.2.2 main

In [ ]:
%%writefile ./main.py

"""
Entry point script for the Strands Agent Demo.
"""
import os
import shutil
import asyncio
import argparse
from dotenv import load_dotenv
from src.graph.builder import build_graph
from src.utils.strands_sdk_utils import strands_utils

# Load environment variables
load_dotenv()

# Observability
from opentelemetry import trace, context
from src.utils.agentcore_observability import set_session_context, add_span_event

# Import event queue for unified event processing
from src.utils.event_queue import clear_queue

def remove_artifact_folder(folder_path="./artifacts/"):
    """
    ./artifact/ 폴더가 존재하면 삭제하는 함수
    
    Args:
        folder_path (str): 삭제할 폴더 경로
    """
    if os.path.exists(folder_path):
        print(f"'{folder_path}' 폴더를 삭제합니다...")
        try:
            shutil.rmtree(folder_path)
            print(f"'{folder_path}' 폴더가 성공적으로 삭제되었습니다.")
        except Exception as e: print(f"오류 발생: {e}")
    else:
        print(f"'{folder_path}' 폴더가 존재하지 않습니다.")

def _setup_execution():
    """Initialize execution environment"""
    remove_artifact_folder()
    clear_queue()
    print("\n=== Starting Queue-Only Event Stream ===")

def _print_conversation_history():
    """Print final conversation history"""
    print("\n=== Conversation History ===")
    from src.graph.nodes import _global_node_states
    shared_state = _global_node_states.get('shared', {})
    history = shared_state.get('history', [])
    
    if history:
        for hist_item in history:
            print(f"[{hist_item['agent']}] {hist_item['message']}")
    else:
        print("No conversation history found")

async def graph_streaming_execution(payload):
    """Execute full graph streaming workflow using new graph.stream_async method"""

    _setup_execution()
    
    # Get user query from payload
    user_query = payload.get("user_query", "")
    session_id = payload.get("session-id", "default")
    context_token = set_session_context(session_id)
    
    try:
        # Get tracer for main application
        tracer = trace.get_tracer(
            instrumenting_module_name=os.getenv("TRACER_MODULE_NAME", "insight_extractor_agent"),
            instrumenting_library_version=os.getenv("TRACER_LIBRARY_VERSION", "1.0.0")
        )
        with tracer.start_as_current_span("insight_extractor_session") as span:   
            
            # Build graph and use stream_async method
            graph = build_graph()
            
            # Stream events from graph execution
            async for event in graph.stream_async({
                "request": user_query,
                "request_prompt": f"Here is a user request: <user_request>{user_query}</user_request>"
            }):
                yield event
            
            _print_conversation_history()
            print("=== Queue-Only Event Stream Complete ===")
            
            # Add Event
            add_span_event(span, "user_query", {"user-query": str(user_query)}) 
    
    finally:
        context.detach(context_token)

if __name__ == "__main__":
    # Parse command line arguments
    parser = argparse.ArgumentParser(description='Strands Agent Demo')
    parser.add_argument('--user_query', type=str, help='User query for the agent')
    parser.add_argument('--session_id', type=str, default='insight-extractor-1', help='Session ID')
    
    args = parser.parse_args()
    

    #########################
    ## modification START  ##
    #########################
    
    # Use argparse values if provided, otherwise use predefined values
    if args.user_query:
        payload = {
            "user_query": args.user_query,
            "session-id": args.session_id
        }
    else:
        # Use predefined query for testing
        payload = {
            "user_query": "너가 작성할 것은 moon market 의 판매 현황 보고서야. 세일즈 및 마케팅 관점으로 분석을 해주고, 차트 생성 및 인사이트도 뽑아서 pdf 파일로 만들어줘. 분석대상은 './data/Dat-fresh-food-claude.csv' 파일 입니다.",
            "session-id": "insight-extractor-1"
        }

    #########################
    ## modification END    ##
    #########################
    
    remove_artifact_folder()

    # Use full graph streaming execution for real-time streaming with graph structure
    async def run_streaming():
        async for event in graph_streaming_execution(payload):
            strands_utils.process_event_for_display(event)

    asyncio.run(run_streaming())

## 2.3. Display tool defination

In [9]:
%%writefile ./src/utils/strands_sdk_utils.py

import logging
import traceback
import asyncio
from src.utils.bedrock import bedrock_info
from strands import Agent, tool
from strands.models import BedrockModel
from botocore.config import Config
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from datetime import datetime

from strands.agent.agent_result import AgentResult
from strands.types.content import ContentBlock, Message
from strands.multiagent.base import MultiAgentBase, NodeResult, MultiAgentResult, Status

# Simple logger setup
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

class ColoredStreamingCallback(StreamingStdOutCallbackHandler):
    COLORS = {
        'blue': '\033[94m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'red': '\033[91m',
        'purple': '\033[95m',
        'cyan': '\033[96m',
        'white': '\033[97m',
    }

    def __init__(self, color='blue'):
        super().__init__()
        self.color_code = self.COLORS.get(color, '\033[94m')
        self.reset_code = '\033[0m'

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"{self.color_code}{token}{self.reset_code}", end="", flush=True)

class strands_utils():

    @staticmethod
    def get_model(**kwargs):

        llm_type = kwargs["llm_type"]
        cache_type = kwargs["cache_type"]
        enable_reasoning = kwargs["enable_reasoning"]

        if llm_type == "claude-sonnet-3-7":    
            ## BedrockModel params: https://strandsagents.com/latest/api-reference/models/?h=bedrockmodel#strands.models.bedrock.BedrockModel
            llm = BedrockModel(
                model_id=bedrock_info.get_model_id(model_name="Claude-V3-7-Sonnet-CRI"),
                streaming=True,
                max_tokens=8192*5,
                stop_sequences=["\n\nHuman"],
                temperature=1 if enable_reasoning else 0.01, 
                additional_request_fields={
                    "thinking": {
                        "type": "enabled" if enable_reasoning else "disabled", 
                        **({"budget_tokens": 8192} if enable_reasoning else {}),
                    }
                },
                cache_prompt=cache_type, # None/ephemeral/defalut
                #cache_tools: Cache point type for tools
                boto_client_config=Config(
                    read_timeout=900,
                    connect_timeout=900,
                    retries=dict(max_attempts=50, mode="adaptive"),
                )
            )   
        elif llm_type == "claude-sonnet-3-5-v-2":
            ## BedrockModel params: https://strandsagents.com/latest/api-reference/models/?h=bedrockmodel#strands.models.bedrock.BedrockModel
            llm = BedrockModel(
                model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-V-2-Sonnet-CRI"),
                streaming=True,
                max_tokens=8192,
                stop_sequences=["\n\nHuman"],
                temperature=0.01,
                cache_prompt=cache_type, # None/ephemeral/defalut
                #cache_tools: Cache point type for tools
                boto_client_config=Config(
                    read_timeout=900,
                    connect_timeout=900,
                    retries=dict(max_attempts=50, mode="standard"),
                )
            )
        else:
            raise ValueError(f"Unknown LLM type: {llm_type}")

        return llm

    @staticmethod
    def get_agent(**kwargs):

        agent_name, system_prompts = kwargs["agent_name"], kwargs["system_prompts"]
        agent_type = kwargs.get("agent_type", "claude-sonnet-3-7")
        enable_reasoning = kwargs.get("enable_reasoning", False)
        prompt_cache_info = kwargs.get("prompt_cache_info", (False, None)) # (True, "default")
        tools = kwargs.get("tools", None)
        streaming = kwargs.get("streaming", True)

        prompt_cache, cache_type = prompt_cache_info
        if prompt_cache: logger.info(f"{Colors.GREEN}{agent_name.upper()} - Prompt Cache Enabled{Colors.END}")
        else: logger.info(f"{Colors.GREEN}{agent_name.upper()} - Prompt Cache Disabled{Colors.END}")

        llm = strands_utils.get_model(llm_type=agent_type, cache_type=cache_type, enable_reasoning=enable_reasoning)
        llm.config["streaming"] = streaming

        agent = Agent(
            model=llm,
            system_prompt=system_prompts,
            tools=tools,
            callback_handler=None # async iterator로 대체 하기 때문에 None 설정
        )

        return agent

    @staticmethod
    def get_agent_state(agent, key, default_value=None):
      """Strands Agent의 state에서 안전하게 값을 가져오는 메서드"""
      value = agent.state.get(key)
      if value is None: return default_value
      return value

    @staticmethod
    def get_agent_state_all(agent):
        return agent.state.get()

    @staticmethod
    def update_agent_state(agent, key, value):
        agent.state.set(key, value)
        #return agent

    @staticmethod
    def update_agent_state_all(target_agent, source_agent):
        """다른 에이전트의 state를 현재 에이전트에 복사"""
        source_state = source_agent.state.get()
        if source_state:
            for key, value in source_state.items():
                target_agent.state.set(key, value)
        return target_agent

    @staticmethod
    async def process_streaming_response(agent, message):
        callback_reasoning, callback_answer = ColoredStreamingCallback('purple'), ColoredStreamingCallback('white')
        response = {"text": "","reasoning": "", "signature": "", "tool_use": None, "cycle": 0}
        try:
            agent_stream = agent.stream_async(message)
            async for event in agent_stream:
                if "reasoningText" in event:
                    response["reasoning"] += event["reasoningText"]
                    callback_reasoning.on_llm_new_token(event["reasoningText"])
                elif "reasoning_signature" in event:
                    response["signature"] += event["reasoning_signature"]
                elif "data" in event:
                    response["text"] += event["data"]
                    callback_answer.on_llm_new_token(event["data"])
                elif "current_tool_use" in event and event["current_tool_use"].get("name"):
                    response["tool_use"] = event["current_tool_use"]["name"]
                    if "event_loop_metrics" in event:
                        if response["cycle"] != event["event_loop_metrics"].cycle_count:
                            response["cycle"] = event["event_loop_metrics"].cycle_count
                            callback_answer.on_llm_new_token(f' \n## Calling tool: {event["current_tool_use"]["name"]} - # Cycle: {event["event_loop_metrics"].cycle_count}\n')
        except Exception as e:
            logger.error(f"Error in streaming response: {e}")
            logger.error(traceback.format_exc())  # Detailed error logging

        return agent, response

    @staticmethod
    async def process_streaming_response_yield(agent, message, agent_name="coordinator", source=None):
        from src.utils.event_queue import put_event
        callback_reasoning, callback_answer = ColoredStreamingCallback('purple'), ColoredStreamingCallback('white')
        response = {"text": "","reasoning": "", "signature": "", "tool_use": None, "cycle": 0}
        try:

            session_id = "ABC"
            agent_stream = agent.stream_async(message)

            async for event in agent_stream:
                #Strands 이벤트를 AgentCore 형식으로 변환
                agentcore_event = await strands_utils._convert_to_agentcore_event(event, agent_name, session_id, source)
                if agentcore_event: 
                    # Put event in global queue for unified processing
                    put_event(agentcore_event)
                    yield agentcore_event

        except Exception as e:
            logger.error(f"Error in streaming response: {e}")
            logger.error(traceback.format_exc())  # Detailed error logging

    # 툴 사용 ID와 툴 이름 매핑을 위한 클래스 변수
    _tool_use_mapping = {}

    @staticmethod
    async def _convert_to_agentcore_event(strands_event, agent_name, session_id, source=None):
        """Strands 이벤트를 AgentCore 스트리밍 형식으로 변환"""

        base_event = {
            "timestamp": datetime.now().isoformat(),
            "session_id": session_id,
            "agent_name": agent_name,
            "source": source or f"{agent_name}_node",
        }

        # 텍스트 데이터 이벤트
        if "data" in strands_event:
            return {
                **base_event,
                "type": "agent_text_stream",
                "event_type": "text_chunk",
                "data": strands_event["data"],
                "chunk_size": len(strands_event["data"])
            }

        # 도구 사용 이벤트
        elif "current_tool_use" in strands_event:
            tool_info = strands_event["current_tool_use"]
            tool_id = tool_info.get("toolUseId")
            tool_name = tool_info.get("name", "unknown")

            # toolUseId와 tool_name 매핑 저장
            if tool_id and tool_name: strands_utils._tool_use_mapping[tool_id] = tool_name

            return {
                **base_event,
                "type": "agent_tool_stream",
                "event_type": "tool_use",
                "tool_name": tool_name,
                "tool_id": tool_id,
                "tool_input": tool_info.get("input", {})
            }

        # message 래퍼 안의 tool result 처리
        if "message" in strands_event:
            message = strands_event["message"]
            if isinstance(message, dict) and "content" in message and isinstance(message["content"], list):
                for content_item in message["content"]:
                    if isinstance(content_item, dict) and "toolResult" in content_item:
                        tool_result = content_item["toolResult"]
                        tool_id = tool_result.get("toolUseId")

                        # 저장된 매핑에서 툴 이름 찾기
                        tool_name = strands_utils._tool_use_mapping.get(tool_id, "external_tool")
                        output = str(tool_result.get("content", [{}])[0].get("text", "")) if tool_result.get("content") else ""

                        return {
                            **base_event,
                            "type": "agent_tool_stream",
                            "event_type": "tool_result", 
                            "tool_name": tool_name,
                            "tool_id": tool_id,
                            "output": output
                        }

        # 추론 이벤트
        elif "reasoning" in strands_event and strands_event.get("reasoning"):
            return {
                **base_event,
                "type": "agent_reasoning_stream",
                "event_type": "reasoning",
                "reasoning_text": strands_event.get("reasoningText", "")[:200]
            }

        return None

    @staticmethod
    def parsing_text_from_response(response):

        """
        Usage (async iterator x): 
        agent = Agent()
        response = agent(query)
        response = strands_utils.parsing_text_from_response(response)
        """

        output = {}
        if len(response.message["content"]) == 2: ## reasoning
            output["reasoning"] = response.message["content"][0]["reasoningContent"]["reasoningText"]["text"]
            output["signature"] = response.message["content"][0]["reasoningContent"]["reasoningText"]["signature"]

        output["text"] = response.message["content"][-1]["text"]

        return output  

    #########################
    ## modification STRART ##
    #########################

    @staticmethod
    def process_event_for_display(event):
        """Process events for colored terminal output"""
        # Initialize colored callbacks for terminal display
        callback_default = ColoredStreamingCallback('white')
        callback_reasoning = ColoredStreamingCallback('cyan')        
        callback_tool = ColoredStreamingCallback('yellow')

        if event:
            if event.get("event_type") == "text_chunk":
                callback_default.on_llm_new_token(event.get('data', ''))

            elif event.get("event_type") == "reasoning":
                callback_reasoning.on_llm_new_token(event.get('reasoning_text', ''))

            elif event.get("event_type") == "tool_use": 
                pass

            elif event.get("event_type") == "tool_result":
                tool_name = event.get("tool_name", "unknown")
                output = event.get("output", "")
                print(f"\n[TOOL RESULT - {tool_name}]", flush=True)

                # Parse output based on function name
                if tool_name == "python_repl_tool" and len(output.split("||")) == 3:
                    status, code, stdout = output.split("||")
                    callback_tool.on_llm_new_token(f"Status: {status}\n")

                    if code: callback_tool.on_llm_new_token(f"Code:\n```python\n{code}\n```\n")
                    if stdout and stdout != 'None': callback_tool.on_llm_new_token(f"Output:\n{stdout}\n")

                elif tool_name == "bash_tool" and len(output.split("||")) == 2:
                    cmd, stdout = output.split("||")
                    if cmd: callback_tool.on_llm_new_token(f"CMD:\n```bash\n{cmd}\n```\n")
                    if stdout and stdout != 'None': callback_tool.on_llm_new_token(f"Output:\n{stdout}\n")

                elif tool_name == "file_read":
                    # file_read 결과는 보통 길어서 앞부분만 표시
                    truncated_output = output[:500] + "..." if len(output) > 500 else output
                    callback_tool.on_llm_new_token(f"File content preview:\n{truncated_output}\n")

                else: # 기타 모든 툴 결과 표시, 코더 툴, 리포터 툴 결과도 다 출력 (for debug)
                    callback_tool.on_llm_new_token(f"Output: pass - you can see that in debug mode\n")
                    #callback_default.on_llm_new_token(f"Output: {output}\n")
                    #pass

    #########################
    ## modification END    ##
    #########################

class FunctionNode(MultiAgentBase):
    """Execute deterministic Python functions as graph nodes."""

    def __init__(self, func, name: str = None):
        super().__init__()
        self.func = func
        self.name = name or func.__name__

    def __call__(self, task=None, **kwargs):
        """Synchronous execution for compatibility with MultiAgentBase"""
        # Pass task and kwargs directly to function
        if asyncio.iscoroutinefunction(self.func): 
            return asyncio.run(self.func(task=task, **kwargs))
        else: 
            return self.func(task=task, **kwargs)

    # Execute function and return standard MultiAgentResult
    async def invoke_async(self, task=None, **kwargs):
        # Execute function (nodes now use global state for data sharing)  
        # Pass task and kwargs directly to function
        if asyncio.iscoroutinefunction(self.func): response = await self.func(task=task, **kwargs)
        else: response = self.func(task=task, **kwargs)

        agent_result = AgentResult(
            stop_reason="end_turn",
            message=Message(role="assistant", content=[ContentBlock(text=str(response["text"]))]),
            metrics={},
            state={}
        )

        # Return wrapped in MultiAgentResult
        return MultiAgentResult(
            status=Status.COMPLETED,
            results={self.name: NodeResult(result=agent_result)}
        )  

Overwriting ./src/utils/strands_sdk_utils.py


## 3. Streaming mechanism
- 노드 레벨 스트리밍, 그래프 레벨 스트리밍 (이건 스트랜즈 지원안함. 랭그래프도 안하는 걸로 알고 있음 )

## 4. AgentCore